<a href="https://colab.research.google.com/github/Raniasoudana/Emoji/blob/main/emojifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install -U -q PyDrive

zip_id1 = '1Gs9ljlmQktWp6BMe-Qu50xTx7YuaKeAY'   

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import zipfile, os

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

if not os.path.exists('MODEL'):
    os.makedirs('MODEL')

# DOWNLOAD ZIP
print ("Downloading zip file")
myzip1 = drive.CreateFile({'id': zip_id1})
myzip1.GetContentFile('model.zip')

# UNZIP ZIP
print ("Uncompressing zip file")
zip_ref = zipfile.ZipFile('model.zip', 'r')
zip_ref.extractall('')
zip_ref.close()


    100% |████████████████████████████████| 993kB 18.5MB/s 
Uncompressing zip file


In [ ]:
!python setup.py install

running install
running bdist_egg
running egg_info
creating emoji.egg-info
writing emoji.egg-info/PKG-INFO
writing dependency_links to emoji.egg-info/dependency_links.txt
writing requirements to emoji.egg-info/requires.txt
writing top-level names to emoji.egg-info/top_level.txt
writing manifest file 'emoji.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'emoji.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/emoji
copying emoji/core.py -> build/lib/emoji
copying emoji/unicode_codes.py -> build/lib/emoji
copying emoji/__init__.py -> build/lib/emoji
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/emoji
copying build/lib/emoji/core.py -> build/bdist.linux-x86_64/egg/emoji
copying build/lib/emoji/unicode_codes.py -> build/bdist.linux-x86_64/egg/emoji
copying build/lib/emoji/__

In [ ]:
!pip install -U -q PyDrive

zip_id1 = '1m7oZi7DYZyODwC0898GwKlwPlCK-FP0R'   #1000 images

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import zipfile, os

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

if not os.path.exists('MODEL'):
    os.makedirs('MODEL')

# DOWNLOAD ZIP
print ("Downloading zip file")
myzip1 = drive.CreateFile({'id': zip_id1})
myzip1.GetContentFile('model.zip')

# UNZIP ZIP
print ("Uncompressing zip file")
zip_ref = zipfile.ZipFile('model.zip', 'r')
zip_ref.extractall('MODEL/')
zip_ref.close()




Uncompressing zip file


In [ ]:
# _*_ coding:utf-8 _*_
import csv
import numpy as np
import emoji
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

def read_glove_vecs(glove_file):
    with open(glove_file, 'r',encoding='UTF-8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

def softmax(x):
#     """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()


def read_csv(filename = 'data/emojify_data.csv'):
    phrase = []
    emoji = []

    with open (filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile)

        for row in csvReader:
            phrase.append(row[0])
            emoji.append(row[1])

    X = np.asarray(phrase)
    Y = np.asarray(emoji, dtype=int)

    return X, Y

def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y


emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":smile:",
                    "3": ":disappointed:",
                    "4": ":fork_and_knife:"}

def label_to_emoji(label):
#     """
#     Converts a label (int or string) into the corresponding emoji code (string) ready to be printed
#     """
    return emoji.emojize(emoji_dictionary[str(label)], use_aliases=True)
              
               


In [ ]:
import numpy as np
import emoji
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
X_data, Y_data = read_csv('mytrain.csv')


X_train, Y_train = X_data[0:127],Y_data[0:127]

X_test, Y_test = X_data[127:],Y_data[127:]

In [ ]:
maxLen = len(max(X_train, key=len).split())

In [ ]:
index = 64
print(X_train[index], label_to_emoji(Y_train[index]))

my dog just had a few puppies ❤️


In [ ]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('MODEL/glove.6B.50d.txt')

In [ ]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)

Using TensorFlow backend.


In [ ]:
# GRADED FUNCTION: sentences_to_indices

def sentences_to_indices(X, word_to_index, max_len):
#     """
#     Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
#     The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
#     Arguments:
#     X -- array of sentences (strings), of shape (m, 1)
#     word_to_index -- a dictionary containing the each word mapped to its index
#     max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
#     Returns:
#     X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
#     """
    
    m = X.shape[0]                                   # number of training examples
    
    ### START CODE HERE ###
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m,max_len))
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words =[word.lower() for word in X[i].split()]
        
        # Initialize j to 0
        j = 0
        
        # Loop over the words of sentence_words
        for w in sentence_words:
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            X_indices[i, j] = word_to_index[w]
            # Increment j to j + 1
            j = j+1
            
    ### END CODE HERE ###
    
    return X_indices

In [ ]:
X1 = np.array(["funny lol", "lets play baseball", "food is ready for you"])
X1_indices = sentences_to_indices(X1,word_to_index, max_len = 5)
print("X1 =", X1)
print("X1_indices =", X1_indices)

X1 = ['funny lol' 'lets play baseball' 'food is ready for you']
X1_indices = [[155345. 225122.      0.      0.      0.]
 [220930. 286375.  69714.      0.      0.]
 [151204. 192973. 302254. 151349. 394475.]]


In [ ]:
# GRADED FUNCTION: pretrained_embedding_layer

def pretrained_embedding_layer(word_to_vec_map, word_to_index):
#     """
#     Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
#     Arguments:
#     word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
#     word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

#     Returns:
#     embedding_layer -- pretrained layer Keras instance
#     """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    ### START CODE HERE ###
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len,emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    ### END CODE HERE ###

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [ ]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])

Instructions for updating:
Colocations handled automatically by placer.
weights[0][1][3] = -0.3403


In [ ]:
# GRADED FUNCTION: Emojify_V2

def Emojify_V2(input_shape, word_to_vec_map, word_to_index):
#     """
#     Function creating the Emojify-v2 model's graph.
    
#     Arguments:
#     input_shape -- shape of the input, usually (max_len,)
#     word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
#     word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

#     Returns:
#     model -- a model instance in Keras
#     """
    
    ### START CODE HERE ###
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(input_shape, dtype='int32')
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)   
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = LSTM(128, return_sequences=True)(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128, return_sequences=False)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(5)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices, outputs=X)
    
    ### END CODE HERE ###
    
    return model

In [ ]:
model = Emojify_V2((maxLen,), word_to_vec_map, word_to_index)
model.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 10, 50)            20000050  
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 128)           91648     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
___________________________

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 5)

In [ ]:
model.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 32, shuffle=True)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
127/127 [==============================] - 2s 17ms/step - loss: 1.6092 - acc: 0.2126
Epoch 2/50
127/127 [==============================] - 0s 2ms/step - loss: 1.5127 - acc: 0.3150
Epoch 3/50
127/127 [==============================] - 0s 2ms/step - loss: 1.5048 - acc: 0.2913
Epoch 4/50
127/127 [==============================] - 0s 2ms/step - loss: 1.4128 - acc: 0.4173
Epoch 5/50
127/127 [==============================] - 0s 2ms/step - loss: 1.3486 - acc: 0.4803
Epoch 6/50
127/127 [==============================] - 0s 2ms/step - loss: 1.2357 - acc: 0.6220
Epoch 7/50
127/127 [==============================] - 0s 2ms/step - loss: 1.0965 - acc: 0.6220
Epoch 8/50
127/127 [==============================] - 0s 2ms/step - loss: 0.9833 - acc: 0.6220
Epoch 9/50
127/127 [==============================] - 0s 2ms/step - loss: 0.8201 - acc: 0.7323
Epoch 10/50
127/127 [==============================] - 0s 2ms/step - loss: 0.7848 - acc: 0.6772


In [ ]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
Y_test_oh = convert_to_one_hot(Y_test, C = 5)
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print()
print("Test accuracy = ", acc)

61/61 [==============================] - 0s 4ms/step

Test accuracy =  0.672131133861229


In [ ]:
x_test = np.array(['Feeling happy'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
print(x_test[0] +' '+  label_to_emoji(np.argmax(model.predict(X_test_indices))))